Generate questions for AWS SA exam

In [22]:
import requests
from dotenv import load_dotenv
import pymongo
import time
import uuid
import sys
import os
import importlib
import re
import json
import csv

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Get the path to the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import const
#importlib.reload(const)    #if we update the file const.py

In [23]:
load_dotenv() 
GENERATIVE_URI = os.environ['GENERATIVE_URI']

In [24]:
db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']
collection = db['tb_pmp']

In [52]:
TOTAL_QUESTIONS = 180
DOMAINS = [
    # {
    #     "name": "People",
    #     "percent": 42,
    #     "num": 20
        
    # },
    # {
    #     "name": "People",
    #     "num": 20
        
    # },
    {
        "name": "Process",
        "num": 20
    },
    {
        "name": "Process",
        "num": 20
    },
    # {
    #     "name": "Business",
    #     "percent": 8,
    #     "num": 20
    # }
]

In [48]:
#Send a POST request
def post_request_generative_ai(text_prompt):
    HEADER = {'Content-Type': 'application/json'}
    json_data = {
        "contents": [
            { "parts": [
                {"text": text_prompt + " Please provide a response in a structured JSON format with the key name 'questions', including all explanations for each answer as a JSON object. Answer and explanation should have the JSON format with keys 'A', 'B', 'C', 'D' or 'E'. Each explanation has more than 50 words."}]
                #{"text": text_prompt + " Please provide a plain string response."}]
            }
        ]
    }

    try:
        r = requests.post(GENERATIVE_URI, headers=HEADER, json=json_data)
        return r.json()
    except Exception as e:
       print(e)
       return {'error': e}

In [41]:
#insert questions first
def insert_questions(json_question):
    if 'question' in json_question:
        existed_doc = collection.find_one({'question': json_question['question']})
        if existed_doc == None:
            #insert new document
            collection.insert_one(json_question)
            #print('Inserted new doc')

In [28]:
#map A - D to 1 - 4
def map_index(a_char):
    if a_char == 'A':
        return 1
    if a_char == 'B':
        return 2
    if a_char == 'C':
        return 3
    if a_char == 'D':
        return 4

In [29]:
def parse_candidate_content(data):
    """
    Parses the content from a dictionary in the 'candidates' list.
    Specifically looks for JSON content within the 'text' part
    and attempts to load it.

    Args:
        data (dict): A dictionary representing an item in the 'candidates' list.

    Returns:
        dict or str or None: If JSON content is found and successfully
                             parsed, it returns the parsed dictionary.
                             If no JSON is found, it returns the raw text.
                             Returns None if the expected structure is not found.
    """
    if not isinstance(data, dict) or 'content' not in data or not isinstance(data['content'], dict) or 'parts' not in data['content'] or not isinstance(data['content']['parts'], list):
        return None

    for part in data['content']['parts']:
        if isinstance(part, dict) and 'text' in part:
            text_content = part['text'].strip()
            # Use regex to find JSON blocks within the text
            json_match = re.search(r'```json\n(.*?)\n```', text_content, re.DOTALL)
            if json_match:
                try:
                    return json.loads(json_match.group(1))
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                    return text_content  # Return the raw text in case of an error
            elif text_content:
                return text_content  # Return the raw text if no JSON block is found

    return None

def extract_questions_from_candidates(response_data):
    """
    Extracts and parses the 'questions' list from the 'candidates'
    in the given response data.

    Args:
        response_data (dict): The dictionary containing the response data.

    Returns:
        list or None: A list of question dictionaries if found and parsed,
                     otherwise None.
    """
    if not isinstance(response_data, dict) or 'candidates' not in response_data or not isinstance(response_data['candidates'], list):
        return None

    for candidate in response_data['candidates']:
        parsed_content = parse_candidate_content(candidate)
        if isinstance(parsed_content, dict) and 'questions' in parsed_content and isinstance(parsed_content['questions'], list):
            return parsed_content['questions']

    return None

# Example usage with your provided data:
#response_data = {'candidates': [{'content': {'parts': [{'text': '```json\n{\n  "questions": [\n    {\n      "question": "Your company is launching a new e-commerce platform on AWS.  The platform will handle sensitive customer data, including credit card information and personally identifiable information (PII). You need to design a secure architecture that meets PCI DSS compliance requirements.  Describe a comprehensive approach to securing the application, covering data at rest, data in transit, and access control. Consider the use of specific AWS services and explain your rationale for choosing them.  Focus on practical implementation details rather than just naming services.",\n      "topics": ["Data Security", "PCI DSS Compliance", "IAM", "KMS", "VPC", "Security Groups", "WAF", "S3", "Encryption"],\n      "difficulty": "Hard"\n    },\n    {\n      "question": "A client has an existing on-premises application that needs to be migrated to AWS. The application interacts with a legacy database that contains highly sensitive customer records. During the migration, you must ensure that the database remains secure and complies with data sovereignty regulations for Europe (GDPR).  How would you design the migration strategy to minimize downtime and maintain security throughout the process?  Be specific about your choices of AWS services and how you will address data encryption, network security, and compliance.  Consider potential challenges and mitigation strategies.",\n      "topics": ["Data Sovereignty", "GDPR", "Database Migration", "VPN", "Direct Connect", "RDS", "Database Encryption", "IAM Roles", "Network Security", "Disaster Recovery"],\n      "difficulty": "Medium"\n    }\n  ]\n}\n```\n'}], 'role': 'model'}, 'finishReason': 'STOP', 'avgLogprobs': -0.3347730217091848}], 'usageMetadata': {'promptTokenCount': 55, 'candidatesTokenCount': 341, 'totalTokenCount': 396, 'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 55}], 'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 341}]}, 'modelVersion': 'gemini-1.5-flash'}

In [ ]:
def generate_list_of_questions():
    no_of_questions = 0
    for domain in DOMAINS:
        #domain['num'] = 15 #for testing
        text_prompt = "The PMP exam format contains questions that are designed to assess the candidate's knowledge and application of project management principles and practices in the real world. The question types include: 'Knowledge-based questions: These questions test the candidate\'s understanding of project management concepts, models, artifacts, and methods. Scenario-based questions: These questions present real-world project management situations and require the candidate to apply their knowledge to determine the appropriate course of action. The challenge in this type of question is that the options provided may seem to be confusing, and they may give you the feeling of more than one correct option. Math-based questions: These questions involve calculations related to project management processes, such as earned value management, estimation techniques, and risk analysis. Professional responsibility questions: Questions that should be answered based on the code of conduct since you are a certified professional. Methodology covers either agile or waterfall or hybrid.'. PMP exam has 10 knowledge areas: Integration Management, Scope Management, Schedule Management, Cost Management, Quality Management, Resource Management, Communication Management, Risk Management, Procurement Management, Stakeholder Management. Generate "+str(domain['num'])+" questions with answers and explanations for the domain "+domain['name']+". There are 3 question types: multiple-choice (80% of total questions), multiple-selection (15% of the total questions), and fill-the-blank (5% of the total questions). Each question has more than 60 words in length and should focus more on complex real-world scenarios rather than definitions."
        raw_generated_text = post_request_generative_ai(text_prompt)
        #print('raw_generated_text')
        questions = extract_questions_from_candidates(raw_generated_text)
        if questions:
            #parse questions and answers
            for q in questions:
                q['domain'] = domain['name']
                q['exported'] = 0
                q['uuid'] = const.generate_random_uuid()
                #print(q)
                insert_questions(q)
                no_of_questions += 1
        else:
            #print(raw_generated_text)
            print("No questions found in the parsed content: " + domain['name'])
        #print(domain['name'] + ': ' + str(no_of_questions))
#test
for i in range(5):
    generate_list_of_questions()    #20 questions ~ 30 secs
    print('===== Finish loop: ' + str(i))

Error decoding JSON: Expecting ',' delimiter: line 105 column 4 (char 10180)
No questions found in the parsed content: Process
Error decoding JSON: Expecting ',' delimiter: line 85 column 5 (char 8408)
No questions found in the parsed content: Business
===== Finish loop: 0
Error decoding JSON: Expecting value: line 49 column 5 (char 5155)
No questions found in the parsed content: Process
Error decoding JSON: Expecting value: line 62 column 5 (char 6464)
No questions found in the parsed content: Business
===== Finish loop: 1
Error decoding JSON: Expecting ',' delimiter: line 79 column 5 (char 7899)
No questions found in the parsed content: Business
===== Finish loop: 2
Error decoding JSON: Expecting value: line 60 column 5 (char 5764)
No questions found in the parsed content: Process
Error decoding JSON: Expecting ',' delimiter: line 85 column 5 (char 7727)
No questions found in the parsed content: Business
===== Finish loop: 3
Error decoding JSON: Expecting value: line 62 column 5 (cha

In [ ]:
#export to CSV file, each file has 65 questions
#Question,Question Type,Answer Option 1,Explanation 1,Answer Option 2,Explanation 2,Answer Option 3,Explanation 3,Answer Option 4,Explanation 4,Answer Option 5,Explanation 5,Answer Option 6,Explanation 6,Correct Answers,Overall Explanation,Domain
def export_csv(filename):
    #get questions that not exported yet. Note that: each part must follow by domain percents
    file_data = []
    #append header line
    file_data.append(['Question','Question Type','Answer Option 1','Explanation 1','Answer Option 2','Explanation 2','Answer Option 3','Explanation 3','Answer Option 4','Explanation 4','Answer Option 5','Explanation 5','Answer Option 6','Explanation 6','Correct Answers','Overall Explanation','Domain'])
        
    for domain in DOMAINS:
        #todo: get random questions
        pipeline = [
            {"$match": {'exported': 0, 'domain': domain['name']}},
            {"$sample": {"size": domain['num']}}
        ]
        random_documents = list(collection.aggregate(pipeline))
        #print(random_documents)
        for doc in random_documents:
            file_data.append([doc['question'].replace('  ', ' ').replace('\n', ''), 'multiple-choice', 
                              doc['options']['A'], doc['explanation']['A'],     #A
                              doc['options']['B'], doc['explanation']['B'],     #B
                              doc['options']['C'], doc['explanation']['C'],     #C
                              doc['options']['D'], doc['explanation']['D'],     #D
                              '', '',   #5
                              '', '',   #6
                              map_index(doc['answer']), #correct answer
                              '', #overall
                              doc['domain'] #domain
                              ])
            #update this doc is exported
            collection.update_one({'uuid': doc['uuid']}, {'$set': {'exported': 1, 'filename': filename}})
    #save all questions to csv
    try:
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(file_data)
        print(f"Data successfully saved to '{filename}'")
    except Exception as e:
        print(f"An error occurred while saving the array: {e}")
    #
#test
#for i in range(2,6):
    #export_csv('aws_sa_test_'+str(i+1)+'_20250504.csv')